# Evaluating a new model on existing responses

In the following eval, we are going to compare how a new model (gpt-4.1-mini) compares to our old model (gpt-4o-mini) by evaluating it on some stored responses. The benefit of this is for most developers, they won't have to spend any time putting together a whole eval -- all of their data will already be stored in their [logs page](https://platform.openai.com/logs).

In [68]:
import openai
import os

client = openai.OpenAI()

We want to see how gpt-4.1 compares to gpt-4o on explaining a code base. Since can only use the responses datasource if you already have user traffic, we're going to generate some example traffic using 4o, and then compare how it does to gpt-4.1. 

We're going to get some example code files from the OpenAI SDK, and ask gpt-4o to explain them to us.

In [ ]:
openai_sdk_file_path = os.path.dirname(openai.__file__)

# Get some example code files from the OpenAI SDK
file_paths   = [
    os.path.join(openai_sdk_file_path, "resources", "evals", "evals.py"),
    os.path.join(openai_sdk_file_path, "resources", "responses", "responses.py"),
    os.path.join(openai_sdk_file_path, "resources", "images.py"),
    os.path.join(openai_sdk_file_path, "resources", "embeddings.py"),
    os.path.join(openai_sdk_file_path, "resources", "files.py"),
]

print(file_paths[0])


Now, lets generate some responses

In [66]:
for file_path in file_paths:
    response = client.responses.create(
        input=[
            {"role": "user",
            "content": [
                {
                    "type": "input_text",
                    "text": "What does this file do?"
                },
                {
                    "type": "input_text",
                    "text": open(file_path, "r").read(),
                },
            ]},
        ],
        model="gpt-4o-mini",
    )
    break
    print(response.id)

In [49]:
grader_system_prompt = """
We've created a consumer-facing Evals product to help AI integrators quickly and clearly understand their models' real-world performance. Your role is to serve as a Universal Evaluator, automatically grading responses to measure how well each model output addresses user needs and expectations.

Given the conversation messages, assign a quality score in the `result` key of the response in the inclusive range between 1.0 (poor) and 7.0 (excellent). Customers will analyze your collective scores and reasoning to gain actionable insights into their models' performance.

---

## Things to Consider

- Evaluate the overall value provided to the user
-  Verify all claims and do not take the AI's statements at face value! Errors might be very hard to find and well hidden.
- Differentiate between minor errors (slight utility reduction) and major errors (significant trust or safety impact).
- Reward answers that closely follow user instructions.
- Reserve the highest and lowest reward scores for cases where you have complete certainty about correctness and utility.


---

## Secondary Labels to Support Final Utility Score Prediction

To help you assign an accurate final utility score, first analyze and predict several important aspects of the AI response. Crucially, these intermediate evaluations should precede your final utility score prediction.

Your structured output must match the provided schema:

- `steps`: A JSON array of objects, each containing:
  - `description`: A detailed explanation of your reasoning for each step.
  - `result`: The float score reached based on the reasoning in this step.

### Steps to Predict (in order):

1. **major_errors**
   - *description*: Identify and explain any significant errors.
   - *conclusion*: List major errors found, or indicate "None".

2. **minor_errors**
   - *description*: Identify and explain any minor inaccuracies.
   - *conclusion*: List minor errors found, or indicate "None".

3. **potential_improvements**
   - *description*: Suggest enhancements that would improve the response.
   - *conclusion*: List suggested improvements, or indicate "None".

---

## JSON Response Structure

Once you predicted all the above fields you need to assign a float between 1 and 7 to indicate the response's utility compared to the alternative responses. Use your best judgment for the meaning of `final_score`. Your response should be a JSON that can be loaded with json.loads in Python and contains:
- steps: An array of objects representing your reasoning steps. Each step includes:
  - description (string): Detailed reasoning for this step.
  - result (string): The float score derived from this reasoning.
- result (float): A numeric quality score as a string, in the inclusive range [1,7].

---

## Notes

- Be meticulous in identifying errors, especially subtle or high-impact ones.
- Avoid being too kind by giving overly high scores easily, it's important to often keep a gap at the top to continue having signal for improvement. Only use [6.5, 7) if the answer is truly mind blowing and you don't see how it could have been improved.
- Never take the AI's responses at face value—verify everything thoroughly.
"""
user_input_message = """**User input**

{{item.input}}

**Response to evaluate**

{{sample.output_text}
"""

In [71]:
logs_eval = client.evals.create(
    name="Code QA Eval",
    data_source_config={
        "type": "logs",
    },
    testing_criteria=[
        {
			"type": "score_model",
            "name": "General Evaluator",
            "model": "o3",
            "input": [{
                "role": "system",
                "content": grader_system_prompt,
            }, {
                "role": "user",
                "content": user_input_message,
            },
            ],
            "pass_threshold": 0.5
        }
    ]
)

First, lets kick off a run to evaluate how good the original responses were. To do this, we just set the filters for what responses we want to evaluate on

In [72]:
gpt_4o_run = client.evals.runs.create(
    eval_id=logs_eval.id,
    data_source={
        "type": "responses",
        "source": {"type": "responses", "limit": len(file_paths)},
    },
)

Now, let's see how 4.1-mini does!

In [73]:
gpt_41_run = client.evals.runs.create(
    eval_id=logs_eval.id,
    data_source={
        "type": "responses",
        "source": {"type": "responses", "limit": len(file_paths)},
        "input_messages": {
            "type": "item_reference",
            "item_reference": "item.input",
        },
        "model": "gpt-4.1-mini",
    }
)

Now, lets go to the dashboard to see how we did!

In [74]:
gpt_41_run.report_url

'https://platform.openai.com/evaluations/eval_6822dd1c4d308190855601dc7fe982bd?project_id=proj_YFvXZA3VxHbZx5aSJv6SAM5B&run_id=evalrun_6822dd1f89b88190a3287c706fd599f4'